In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [9]:
RUG = pd.read_pickle("../obfuscated_data.pkl")
RUG.interpolate(method='linear', inplace=True, limit=20)
RUG = RUG[::10]
dfs = [RUG.filter([i]) for i in RUG]

In [10]:
RUG

,Location 1,Location 2,Location 3,Location 4,Location 5,Location 6,Location 7,Location 8,Location 9,Location 10,Location 11,Location 12,Location 13
date,,,,,,,,,,,,,
2017-01-01 00:00:00+01:00,254.539382,1.030,NaN,2.02,0.51,87.180074,70.544564,0.737664,54.730393,80.548130,54.500049,0.0,65.575694
2017-01-01 00:10:00+01:00,238.141130,1.030,NaN,2.02,0.51,87.186223,70.523249,-0.754613,54.736392,80.019617,54.454766,0.0,65.544464
2017-01-01 00:20:00+01:00,305.379914,1.030,NaN,2.02,0.51,87.192372,70.501934,-0.260751,54.742390,79.509301,54.440305,0.0,65.090878
2017-01-01 00:30:00+01:00,329.373055,1.030,NaN,2.02,0.51,87.198520,70.480618,-0.182155,54.748388,80.051132,54.429470,0.0,64.956293
2017-01-01 00:40:00+01:00,279.974483,1.030,NaN,2.02,0.51,87.204669,70.459303,-0.025453,54.754387,79.873456,54.418635,0.0,65.217987
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 23:10:00+01:00,317.433333,1.176,0.012,3.72,0.24,86.629042,68.844854,0.069574,54.247488,89.345715,54.997689,0.0,67.313597
2022-10-31 23:20:00+01:00,224.912778,0.528,0.000,3.48,0.24,86.630885,69.142068,0.069309,54.255874,90.064817,55.352572,0.0,68.360647
2022-10-31 23:30:00+01:00,182.362125,0.504,0.000,5.28,0.24,86.663622,68.833535,0.069044,54.265009,89.979376,54.972748,0.0,68.897534


# Timeseries LSTM

In [16]:
def func(df, name):

    df2 = df.copy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(df2)

    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    # reshape into X=t and Y=t+1
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.fit(trainX, trainY, epochs=25, verbose=2, callbacks=[callback], batch_size=64)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    # print('Train Score: %.2f RMSE' % (trainScore))
    testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    # print('Test Score: %.2f RMSE' % (testScore))

    return (name, (trainScore, testScore))

In [17]:
results = []

for name, df in zip([i for i in RUG], dfs):
    print(name)
    
    if df.isnull().values.any():
        print("NaN")
        results.append((name, (np.nan, np.nan)))
        print("-------------")
        continue

    r = func(df, name)
    print("-------------")
    results.append(r)

Location 1
Epoch 1/25
3211/3211 - 22s - loss: 0.0070 - mse: 0.0070 - 22s/epoch - 7ms/step
Epoch 2/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 3/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 4/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 5/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 6/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 7/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 8/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 9/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 10/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 11/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 12/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 13/25
3211/3211 - 13s - loss: 0.

In [18]:
for i in results:
    print(i)

('Location 1', (26.03481797558312, 26.036808947232313))
('Location 2', (nan, nan))
('Location 3', (nan, nan))
('Location 4', (0.3543802521474015, 0.6887317161515869))
('Location 5', (0.2071343279799014, 0.40310868598393146))
('Location 6', (0.44801859183426057, 0.33049833457650934))
('Location 7', (0.36961394776702716, 0.5409596566271693))
('Location 8', (nan, nan))
('Location 9', (0.14893674172695118, 0.07912445573885735))
('Location 10', (17.572360847731034, 20.743297684615147))
('Location 11', (34.60993171226959, 22.60890138038569))
('Location 12', (nan, nan))
('Location 13', (0.69313453906051, 0.5847372339931131))


In [20]:
lstm_results = [[i[0], i[1][0], i[1][1]] for i in results]
# lstm_results.append(['Renkum-Heelsum\Renkum|Supplied', np.nan, np.nan])
# lstm_results.append(['Pompstation Wageningseberg\Flow', np.nan, np.nan])
lstm_results = pd.DataFrame(lstm_results)


lstm_results.columns = ["Location", "Train RMSE", "Test RMSE"]
lstm_results.set_index("Location", inplace=True)
lstm_results.sort_index(inplace=True)

print(lstm_results.round(3).to_latex())

\begin{tabular}{lrr}
\toprule
 & Train RMSE & Test RMSE \\
Location &  &  \\
\midrule
Location 1 & 26.035000 & 26.037000 \\
Location 10 & 17.572000 & 20.743000 \\
Location 11 & 34.610000 & 22.609000 \\
Location 12 & NaN & NaN \\
Location 13 & 0.693000 & 0.585000 \\
Location 2 & NaN & NaN \\
Location 3 & NaN & NaN \\
Location 4 & 0.354000 & 0.689000 \\
Location 5 & 0.207000 & 0.403000 \\
Location 6 & 0.448000 & 0.330000 \\
Location 7 & 0.370000 & 0.541000 \\
Location 8 & NaN & NaN \\
Location 9 & 0.149000 & 0.079000 \\
\bottomrule
\end{tabular}

